In [ ]:
%%capture
!pip install hierarchicalforecast
!pip install -U numba statsforecast datasetsforecast

In [ ]:
import numpy as np
import pandas as pd

# obtain hierarchical data
from datasetsforecast.hierarchical import HierarchicalData

# compute base forecast no coherent
from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA, Naive

# obtain hierarchical reconciliation methods and evaluation
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.evaluation import HierarchicalEvaluation
from hierarchicalforecast.methods import BottomUp, TopDown, MiddleOut

/usr/local/lib/python3.8/dist-packages/statsforecast/core.py:21: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In this example we will use the `TourismSmall` dataset. The following cell gets the time series for the different levels in the hierarchy, the summing matrix `S` which recovers the full dataset from the bottom level hierarchy and the indices of each hierarchy denoted by `tags`.

In [ ]:
Y_df, S_df, tags = HierarchicalData.load("./data", "Labour")
Y_df["ds"] = pd.to_datetime(Y_df["ds"])

100%|██████████| 1.30M/1.30M [00:01<00:00, 1.18MiB/s]


In [ ]:
Y_df.head()

,unique_id,ds,y
0,['Total'],1978-02-01,5985.659716
1,['Total'],1978-03-01,6040.560795
2,['Total'],1978-04-01,6054.213859
3,['Total'],1978-05-01,6038.264751
4,['Total'],1978-06-01,6031.342299


In [ ]:
S_df.iloc[:5, :5]

,"['Employed full-time', 'Males', 'New South Wales']","['Employed full-time', 'Males', 'Victoria']","['Employed full-time', 'Males', 'Queensland']","['Employed full-time', 'Males', 'South Australia']","['Employed full-time', 'Males', 'Western Australia']"
['Total'],1.0,1.0,1.0,1.0,1.0
['Australian Capital Territory'],1.0,1.0,1.0,1.0,0.0
['New South Wales'],0.0,0.0,0.0,0.0,1.0
['Northern Territory'],0.0,0.0,0.0,0.0,0.0
['Queensland'],0.0,0.0,0.0,0.0,0.0


In [ ]:
tags

{'Country': array(["['Total']"], dtype=object),
 'Country/Region': array(["['Australian Capital Territory']", "['New South Wales']",
        "['Northern Territory']", "['Queensland']", "['South Australia']",
        "['Tasmania']", "['Victoria']", "['Western Australia']"],
       dtype=object),
 'Country/Gender/Region': array(["['Females', 'Australian Capital Territory']",
        "['Males', 'Australian Capital Territory']",
        "['Females', 'New South Wales']", "['Males', 'New South Wales']",
        "['Females', 'Northern Territory']",
        "['Males', 'Northern Territory']", "['Females', 'Queensland']",
        "['Males', 'Queensland']", "['Females', 'South Australia']",
        "['Males', 'South Australia']", "['Females', 'Tasmania']",
        "['Males', 'Tasmania']", "['Females', 'Victoria']",
        "['Males', 'Victoria']", "['Females', 'Western Australia']",
        "['Males', 'Western Australia']"], dtype=object),
 'Country/Employment/Gender/Region': array(["['Employed f

We split the dataframe in train/test splits.

In [ ]:
Y_test_df = Y_df.groupby("unique_id").tail(12)
Y_train_df = Y_df.drop(Y_test_df.index)

In [ ]:
Y_test_df = Y_test_df.set_index("unique_id")
Y_train_df = Y_train_df.set_index("unique_id")

The following cell computes the *base forecast* for each time series using the `auto_arima` and `naive` models. Observe that `Y_hat_df` contains the forecasts but they are not coherent.

In [ ]:
%%capture
fcst = StatsForecast(
    df=Y_train_df, models=[AutoARIMA(season_length=12), Naive()], freq="M", n_jobs=-1
)
Y_hat_df = fcst.forecast(h=12)

In [ ]:
# S
np.save(open("S_tensor.npy", "wb"), S_df)
S_df

,"['Employed full-time', 'Males', 'New South Wales']","['Employed full-time', 'Males', 'Victoria']","['Employed full-time', 'Males', 'Queensland']","['Employed full-time', 'Males', 'South Australia']","['Employed full-time', 'Males', 'Western Australia']","['Employed full-time', 'Males', 'Tasmania']","['Employed full-time', 'Males', 'Northern Territory']","['Employed full-time', 'Males', 'Australian Capital Territory']","['Employed full-time', 'Females', 'New South Wales']","['Employed full-time', 'Females', 'Victoria']",...,"['Employed part-time', 'Males', 'Northern Territory']","['Employed part-time', 'Males', 'Australian Capital Territory']","['Employed part-time', 'Females', 'New South Wales']","['Employed part-time', 'Females', 'Victoria']","['Employed part-time', 'Females', 'Queensland']","['Employed part-time', 'Females', 'South Australia']","['Employed part-time', 'Females', 'Western Australia']","['Employed part-time', 'Females', 'Tasmania']","['Employed part-time', 'Females', 'Northern Territory']","['Employed part-time', 'Females', 'Australian Capital Territory']"
['Total'],1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
['Australian Capital Territory'],1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
['New South Wales'],0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
['Northern Territory'],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
['Queensland'],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
['South Australia'],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
['Tasmania'],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
['Victoria'],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
['Western Australia'],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
"['Females', 'Australian Capital Territory']",1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# y_hat
y_i = Y_hat_df.reset_index()
y_hat = pd.pivot(y_i, index="unique_id", columns="ds", values="AutoARIMA").to_numpy()
np.save(open("yhat_tensor.npy", "wb"), y_hat)

In [ ]:
# gt
gt_i = Y_test_df.reset_index()
gt = pd.pivot(gt_i, index="unique_id", columns="ds", values="y").to_numpy()
np.save(open("gt_tensor.npy", "wb"), gt)

In [ ]:
# parent list
node_map = {}
node_map_inv = {}

i = 0

for l in [
    "Country/Employment/Gender/Region",
    "Country/Gender/Region",
    "Country/Region",
    "Country",
]:
    for c in tags[l]:
        node_map[i] = c
        node_map_inv[c] = i
        i += 1

In [ ]:
def findParent(i):
    if i in tags["Country/Employment/Gender/Region"]:
        cut1 = i.find("'")
        cut = i.find(",")
        return i.replace(i[cut1 : cut + 2], "")
    if i in tags["Country/Gender/Region"]:
        cut1 = i.find("'")
        cut = i.find(",")
        return i.replace(i[cut1 : cut + 2], "")
    if i in tags["Country/Region"]:
        return "['Total']"

In [ ]:
parent_list = []

for i, ro in S_df.iterrows():
    if i in tags["Country/Employment/Gender/Region"]:
        lv3 = i
        lv2 = findParent(i)
        lv1 = findParent(lv2)
        lv0 = findParent(lv1)
        parent_list.append(
            [node_map_inv[lv3], node_map_inv[lv2], node_map_inv[lv1], node_map_inv[lv0]]
        )
    elif i in tags["Country/Gender/Region"]:
        lv2 = i
        lv1 = findParent(lv2)
        lv0 = findParent(lv1)
        parent_list.append(
            [node_map_inv[lv2], node_map_inv[lv1], node_map_inv[lv0], -1]
        )
    elif i in tags["Country/Region"]:
        lv1 = i
        lv0 = findParent(lv1)
        parent_list.append([node_map_inv[lv1], node_map_inv[lv0], -1, -1])
    else:
        parent_list.append([node_map_inv[i], -1, -1, -1])
parent_list = sorted(parent_list, key=lambda x: x[0])

In [ ]:
np.save(open("parent.npy", "wb"), np.array(parent_list))

In [ ]:
# p
top_down_tensor = np.zeros_like(gt)
for l in parent_list:
    i = l[0]
    for c in [l[3], l[2], l[1]]:
        if c != -1:
            break
    top_down_tensor[i, :] = gt[i, :] / (gt[c, :] + 1e-9)

In [ ]:
np.save(open("top_down_tensor.npy", "wb"), top_down_tensor)

In [ ]:
level_2_tensor = np.zeros_like(gt)

for l in parent_list:
    nz = list(filter(lambda x: x != -1, l))
    if len(nz) == 1:
        continue
    i = l[0]
    c = l[-2]
    level_2_tensor[i, :] = gt[i, :] / (gt[c, :] + 1e-9)

In [ ]:
np.save(open("level_2_tensor.npy", "wb"), level_2_tensor)

In [ ]:
level_3_tensor = np.zeros_like(gt)

for l in parent_list:
    nz = list(filter(lambda x: x != -1, l))
    if len(nz) <= 2:
        continue
    i = l[0]
    c = l[-3]
    level_3_tensor[i, :] = gt[i, :] / (gt[c, :] + 1e-9)
level_3_tensor

array([[ 1.95620604,  1.92089762,  1.75787739,  1.80363892,  1.75030706,
         1.76292152,  1.83109355,  1.71981919,  1.86775755,  1.7253433 ,
         1.76785389,  1.74869609],
       [ 0.58636013,  0.58184577,  0.54634408,  0.55757086,  0.54870665,
         0.52799941,  0.55394843,  0.51489695,  0.56294892,  0.52218267,
         0.52837164,  0.52938336],
       [ 9.67686866,  9.7833044 ,  9.44104919,  9.77558881,  9.62086664,
         9.57760809,  9.92658179,  9.58465856,  9.52374515,  9.70343759,
         9.49994306,  9.78507117],
       [ 0.39718406,  0.39562143,  0.38335335,  0.39018675,  0.38107369,
         0.38463317,  0.37259906,  0.37362111,  0.38683761,  0.38679403,
         0.38856402,  0.40493119],
       [ 0.34335333,  0.34560128,  0.34609403,  0.33835119,  0.34000615,
         0.33952163,  0.34440636,  0.34528797,  0.34456065,  0.34545087,
         0.34696493,  0.35144592],
       [ 0.09733334,  0.0971098 ,  0.10074992,  0.10044318,  0.1010543 ,
         0.1001461 ,  

In [ ]:
np.save(open("level_3_tensor.npy", "wb"), level_3_tensor)

In [ ]:
N_CHUNCKS = 8

for t, n in [
    (y_hat, "yhat_tensor"),
    (gt, "gt_tensor"),
    (top_down_tensor, "top_down_tensor"),
    (level_2_tensor, "level_2_tensor"),
    (level_3_tensor, "level_3_tensor"),
]:
    l = np.array_split(t, N_CHUNCKS)
    for i, p in enumerate(l):
        np.save(open("mpi/" + n + "_" + str(i) + ".npy", "wb"), p)

The following cell makes the previous forecasts coherent using the `HierarchicalReconciliation` class. The used methods to make the forecasts coherent are:
- `BottomUp`: The reconciliation of the method is a simple addition to the upper levels.
- `TopDown`: The second method constrains the base-level predictions to the top-most aggregate-level serie and then distributes it to the disaggregate series through the use of proportions. 
- `MiddleOut`: Anchors the base predictions in a middle level.

In [ ]:
reconcilers = [
    BottomUp(),
    TopDown(method="forecast_proportions"),
    MiddleOut(
        middle_level="Country/Gender/Region", top_down_method="forecast_proportions"
    ),
]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, Y_df=Y_train_df, S=S_df, tags=tags)

The `HierarchicalForecast` package includes the `HierarchicalEvaluation` class to evaluate the different hierarchies and also is capable of compute scaled metrics compared to a benchmark model.

In [ ]:
def mse(y, y_hat):
    return np.mean((y - y_hat) ** 2)


evaluator = HierarchicalEvaluation(evaluators=[mse])
evaluation = evaluator.evaluate(
    Y_hat_df=Y_rec_df, Y_test_df=Y_test_df, tags=tags, benchmark="Naive"
)
evaluation.filter(like="ARIMA", axis=1).T

level,Overall,Country,Country/Region,Country/Gender/Region,Country/Employment/Gender/Region
metric,mse-scaled,mse-scaled,mse-scaled,mse-scaled,mse-scaled
AutoARIMA,0.425294,0.054193,0.669191,0.946207,0.806818
AutoARIMA/BottomUp,0.537824,0.235726,0.827911,0.901878,0.806818
AutoARIMA/TopDown_method-forecast_proportions,0.424542,0.054193,0.508586,0.991649,0.922604
AutoARIMA/MiddleOut_middle_level-Country/Gender/Region_top_down_method-forecast_proportions,0.391065,0.054579,0.399931,0.946207,0.877669


### References
- [Orcutt, G.H., Watts, H.W., & Edwards, J.B.(1968). Data aggregation and information loss. The American 
Economic Review, 58 , 773{787)](http://www.jstor.org/stable/1815532).
- [Disaggregation methods to expedite product line forecasting. Journal of Forecasting, 9 , 233–254. 
doi:10.1002/for.3980090304](https://onlinelibrary.wiley.com/doi/abs/10.1002/for.3980090304).<br>
- [An investigation of aggregate variable time series forecast strategies with specific subaggregate 
time series statistical correlation. Computers and Operations Research, 26 , 1133–1149. 
doi:10.1016/S0305-0548(99)00017-9](https://doi.org/10.1016/S0305-0548(99)00017-9).
- [Hyndman, R.J., & Athanasopoulos, G. (2021). "Forecasting: principles and practice, 3rd edition: 
Chapter 11: Forecasting hierarchical and grouped series.". OTexts: Melbourne, Australia. OTexts.com/fpp3 
Accessed on July 2022.](https://otexts.com/fpp3/hierarchical.html)